In [216]:
#importing library
import numpy as np
import pandas as pd
#train_test_split
from sklearn.model_selection import train_test_split
#StandardScaler
from sklearn.preprocessing import StandardScaler
#Regression Model
from sklearn.linear_model import LinearRegression,Lasso,Ridge
#KNeighbors
from sklearn.neighbors import KNeighborsRegressor
#neural_network
from sklearn.neural_network import MLPRegressor
#ensemble
from sklearn.svm import LinearSVR,SVR
#tree
from sklearn.tree import DecisionTreeRegressor
#RandomForest
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
#XGBRegressor
from xgboost import XGBRegressor
#lightgbm
from lightgbm import LGBMRegressor
#catboost
from catboost import CatBoostRegressor
#warnings
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import seaborn as sns

# Loading the Dataset

In [217]:
df=pd.read_csv('/kaggle/input/delhi-house-price-prediction/MagicBricks.csv')
#loading the dataset
df

,Area,BHK,Bathroom,Furnishing,Locality,Parking,Price,Status,Transaction,Type,Per_Sqft
0,800.0,3,2.0,Semi-Furnished,Rohini Sector 25,1.0,6500000,Ready_to_move,New_Property,Builder_Floor,NaN
1,750.0,2,2.0,Semi-Furnished,"J R Designers Floors, Rohini Sector 24",1.0,5000000,Ready_to_move,New_Property,Apartment,6667.0
2,950.0,2,2.0,Furnished,"Citizen Apartment, Rohini Sector 13",1.0,15500000,Ready_to_move,Resale,Apartment,6667.0
3,600.0,2,2.0,Semi-Furnished,Rohini Sector 24,1.0,4200000,Ready_to_move,Resale,Builder_Floor,6667.0
4,650.0,2,2.0,Semi-Furnished,Rohini Sector 24 carpet area 650 sqft status R...,1.0,6200000,Ready_to_move,New_Property,Builder_Floor,6667.0
...,...,...,...,...,...,...,...,...,...,...,...
1254,4118.0,4,5.0,Unfurnished,Chittaranjan Park,3.0,55000000,Ready_to_move,New_Property,Builder_Floor,12916.0
1255,1050.0,3,2.0,Semi-Furnished,Chittaranjan Park,3.0,12500000,Ready_to_move,Resale,Builder_Floor,12916.0
1256,875.0,3,3.0,Semi-Furnished,Chittaranjan Park,3.0,17500000,Ready_to_move,New_Property,Builder_Floor,12916.0
1257,990.0,2,2.0,Unfurnished,Chittaranjan Park Block A,1.0,11500000,Ready_to_move,Resale,Builder_Floor,12916.0


In [218]:
x.isna().sum()

AttributeError: 'tuple' object has no attribute 'isna'

# Preprocessing

In [219]:
def onehot_encode(df,column,rename=False):
    df=df.copy()
    if rename==True:
        df[column]=df[column].replace({x:i for i,x in enumerate(df[column].unique())})
    dummies=pd.get_dummies(df[column],prefix=column)
    df=pd.concat([df,dummies],axis=1)
    df=df.drop(column,axis=1)
    return df

In [220]:
def preprocess_inputs(df):
    df=df.copy()
    #dropping the rows with missing values in per_sqrt column
    df['Bathroom']=df['Bathroom'].fillna(df['Bathroom'].mode()[0])
    df=df.drop('Per_Sqft',axis=1)
    df=df.drop(df[df['Price'].isna()].index,axis=0).reset_index(drop=True)
    for column in ['Type','Parking']:
        df[column]=df[column].fillna(df[column].mode()[0])
    df['Status']=df['Status'].apply(lambda x:0 if x=='Almost_ready' else 1)
    df['Transaction']=df['Transaction'].apply(lambda x:0 if x=='Resale' else 1)
    df['Type']=df['Type'].apply(lambda x:0 if x=='Building_Floor' else 1)
    onehot_column=['Locality','Furnishing']
    df=onehot_encode(df,'Locality',rename=True)
    df=onehot_encode(df,'Furnishing',rename=False)
    y=df['Price']
    x=df.drop('Price',axis=1)
    #train_test_split
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7)
    #scaling
    scaler=StandardScaler()
    scaler.fit(x_train)
    x_train=pd.DataFrame(scaler.transform(x_train),columns=x_train.columns)
    x_test=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns)
    
    return x_train,x_test,y_train,y_test

In [221]:
df['Locality']

0                                        Rohini Sector 25
1                  J R Designers Floors, Rohini Sector 24
2                     Citizen Apartment, Rohini Sector 13
3                                        Rohini Sector 24
4       Rohini Sector 24 carpet area 650 sqft status R...
                              ...                        
1254                                    Chittaranjan Park
1255                                    Chittaranjan Park
1256                                    Chittaranjan Park
1257                            Chittaranjan Park Block A
1258                                    Chittaranjan Park
Name: Locality, Length: 1259, dtype: object

In [222]:
df.isna().sum()

Area             0
BHK              0
Bathroom         2
Furnishing       5
Locality         0
Parking         33
Price            0
Status           0
Transaction      0
Type             5
Per_Sqft       241
dtype: int64

In [223]:
x_train,x_test,y_train,y_test=preprocess_inputs(df)
x_train

,Area,BHK,Bathroom,Parking,Status,Transaction,Type,Locality_0,Locality_1,Locality_2,...,Locality_358,Locality_359,Locality_360,Locality_361,Locality_362,Locality_363,Locality_364,Furnishing_Furnished,Furnishing_Semi-Furnished,Furnishing_Unfurnished
0,2.052489,1.245030,1.405244,0.006566,0.255531,1.276656,0.0,-0.067535,-0.148465,0.0,...,-0.03371,-0.127073,-0.03371,-0.03371,-0.03371,-0.0477,-0.033710,-0.412294,0.856885,-0.613647
1,0.043006,0.214136,-0.515656,-0.154121,0.255531,-0.783296,0.0,-0.067535,-0.148465,0.0,...,-0.03371,-0.127073,-0.03371,-0.03371,-0.03371,-0.0477,-0.033710,-0.412294,-1.167018,1.629602
2,-0.409127,-0.816758,-0.515656,-0.154121,0.255531,1.276656,0.0,-0.067535,-0.148465,0.0,...,-0.03371,-0.127073,-0.03371,-0.03371,-0.03371,-0.0477,-0.033710,-0.412294,-1.167018,1.629602
3,-0.057468,0.214136,-0.515656,-0.154121,0.255531,1.276656,0.0,-0.067535,-0.148465,0.0,...,-0.03371,-0.127073,-0.03371,-0.03371,-0.03371,-0.0477,-0.033710,2.425451,-1.167018,-0.613647
4,1.047748,1.245030,2.365694,0.167253,0.255531,1.276656,0.0,-0.067535,-0.148465,0.0,...,-0.03371,-0.127073,-0.03371,-0.03371,-0.03371,-0.0477,-0.033710,-0.412294,0.856885,-0.613647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,0.913782,1.245030,2.365694,0.167253,0.255531,1.276656,0.0,-0.067535,-0.148465,0.0,...,-0.03371,-0.127073,-0.03371,-0.03371,-0.03371,-0.0477,-0.033710,-0.412294,0.856885,-0.613647
877,-0.894752,0.214136,-0.515656,0.006566,0.255531,-0.783296,0.0,-0.067535,-0.148465,0.0,...,-0.03371,-0.127073,-0.03371,-0.03371,-0.03371,-0.0477,-0.033710,-0.412294,0.856885,-0.613647
878,-0.298606,-0.816758,-0.515656,-0.154121,0.255531,-0.783296,0.0,-0.067535,-0.148465,0.0,...,-0.03371,-0.127073,-0.03371,-0.03371,-0.03371,-0.0477,29.664794,-0.412294,-1.167018,1.629602
879,0.243955,1.245030,-0.515656,-0.154121,0.255531,-0.783296,0.0,-0.067535,-0.148465,0.0,...,-0.03371,-0.127073,-0.03371,-0.03371,-0.03371,-0.0477,-0.033710,2.425451,-1.167018,-0.613647


In [224]:
x_test.isna().sum().sum()

0

In [225]:
{column:len(x[column].unique()) for column in x.select_dtypes('object').columns}

AttributeError: 'tuple' object has no attribute 'select_dtypes'

In [ ]:
len(df[df['Per_Sqft'].isna()].index)+1018

In [ ]:
corr=df.corr()
plt.figure(figsize=(5,5))
sns.heatmap(corr,annot=True,cmap='mako')
plt.show()

* Bathroom and Price has very high correlation

In [226]:
df['Status'].unique()

array(['Ready_to_move', 'Almost_ready'], dtype=object)

# Training the Model

In [227]:
models={
    'Linear Regression':LinearRegression(),
    'Linear Regression (L2 Regularization)':Ridge(),
    'Linear Regression (L1 Regularization)':Lasso(),
    'K-Nearest Neighbors':KNeighborsRegressor(),
    'Neural Network':MLPRegressor(),
    'Support Vector Machine (Linear Kernal)':LinearSVR(),
    'Support Vector Machine(RBF Kernel)':SVR(),
    'Decision Tree':DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting':GradientBoostingRegressor(),
    'XGBoost':XGBRegressor(),
    'LightGBM':LGBMRegressor(),
    'CatBoost':CatBoostRegressor(verbose=0)
}

In [228]:
for name,model in models.items():
    model.fit(x_train,y_train)
    print(name)
    print(model.score(x_test,y_test))

Linear Regression
-4.200352509823493e+24
Linear Regression (L2 Regularization)
0.6982648965235562
Linear Regression (L1 Regularization)
0.6981398428488272
K-Nearest Neighbors
0.520074601331693
Neural Network
-0.6525866506774172
Support Vector Machine (Linear Kernal)
-0.6526633129519155
Support Vector Machine(RBF Kernel)
-0.08246437959972419
Decision Tree
0.7864458920251253
Random Forest
0.8432862699826972
Gradient Boosting
0.8700150813513993
XGBoost
0.8738537961501615
LightGBM
0.7827230040413795
CatBoost
0.8811356097593379
